<a href="https://colab.research.google.com/github/ENV716/Energy_Modeling_F2022/blob/main/Lab/Lab11/Lab11_MoreNetworkModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 11 - More Network Models with Pyomo**


Learning goals for Lab 11:
* Implement other types of Network Models - Max flow;
* Implement other types of Network Models - Shortest path.


## Initializing 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

Mounted at /content/drive


Installing Pyomo and solver. Recall for teh shortest path example we have binary variables so we will need to use another solver. Instead of installing glpk, thsi time we will install COIN-OR CBC. \\ 
COIN-OR CBC is a multi-threaded open-source Coin-or branch and cut **mixed-integer linear programming solver**. CBC is generally a good choice for a general purpose MILP solver for medium to large scale problems.

In [ ]:
!pip install pyomo
#!apt-get install -y -qq glpk-utils
!apt-get install -y -qq coinor-cbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.7 MB 7.7 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcg

Importing pyomo and cbc solver.

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc',executable='/usr/bin/cbc')

## Exercise 1: Maximum Flow Model - Natural Gas company (A9-Q2)

Let's start by writing the max flow problem.

**Sets** \\
$N$: set of nodes \\
$K$: set of nodes without source and sink nodes \\
$A$: set of arcs $(ij)$ \\

**Parameters** \\
$u_{ij}$: flow capacity for arc $(ij)$

**Decision Variable** \\
$x_{ij}$: how much flow on arc $(ij)$ - any value from 0 to $u_{ij}$

**Model** \\
$ max \ \sum_{j \in N} x_{1j}$ \\
$ s.t.$
$ \ \sum_{j \in N} x_{ij} = \sum_{j \in N} x_{ji} \quad \forall i \in K $ \\
$ \quad \quad \sum_{j \in N} x_{1j} = \sum_{j \in N} x_{j10} $ \\
$ \quad \quad x_{ij} \leq u_{ij} \quad \forall (ij) \in A $ \\
$ \quad \quad x_{ij} \geq 0 \quad \forall (ij) \in A $


### Implementing model from A9 question 2.

In [ ]:
model=ConcreteModel()

model.Nodes=Set(initialize=range(1,11))  

model.first=1
model.last=10
#defining set of nodes without origin and destination
model.NodesK=Set(within=model.Nodes,initialize=range(2,10))



model.Arcs=Set(within=model.Nodes*model.Nodes, 
                initialize=[(1,2),(1,3),(1,4),(2,5),(5,2),
                            (3,4),(3,5),(3,6),(5,3),(6,3),
                            (4,6),(4,10),(6,4),(5,7),(5,9),(9,5),
                            (6,7),(6,8),(6,10),(7,6),(8,6),
                            (7,8),(7,9),(8,7),(9,7),(8,9),(8,10),(9,8),(9,10)])

model.FlowCap=Param(model.Arcs,
                     initialize={(1,2):5,(1,3):12,(1,4):8,(2,5):6,(5,2):3,
                            (3,4):2,(3,5):4,(3,6):5,(5,3):3,(6,3):4,
                            (4,6):9,(4,10):2,(6,4):5,(5,7):6,(5,9):5,(9,5):2,
                            (6,7):3,(6,8):6,(6,10):8,(7,6):4,(8,6):3,
                            (7,8):5,(7,9):7,(8,7):2,(9,7):3,(8,9):5,(8,10):7,(9,8):1,(9,10):4})

#Add dec variables
model.x=Var(model.Arcs,domain=NonNegativeReals)

In [ ]:
#Adding objective function


In [ ]:
#Adding constraints
#Flow balance transhipment nodes - for all nodes in K


#printing constraints


In [ ]:
#Adding constraints
#flow origin = flow destination - only one


#printing org = dest constraints


In [ ]:
#Adding constraints
#arc flow capacity


#printing max flow constraints


In [ ]:
#Solving model
opt.solve(model)

#Print results
print("Max Flow from 1 to 10 =",model.maxflow())
print("Decision Variables")
for a in model.Arcs:
    print(model.x[a],model.x[a].value)

## Exercise 2: Shortest Path - min cost (A9-Q1)

Let's start by writing the shortest flow model formulation.

**Sets** \\
$N$: set of nodes \\
$K$: set of nodes without source and sink \\
$A$: set of arcs $(ij)$ \\

**Parameters** \\
$c_{ij}$: cost for using arc $ij$

**Decision Variable** \\
$x_{ij}$: 1 if arc $(ij)$ is being used, 0 o.w. - binary

**Model** \\
$ min \ \sum_{(ij) \in A} c_{ij}*x_{ij}$ \\
$ s.t.$
$ \ \sum_{j \in N} x_{ij} = \sum_{j \in N} x_{ji} \quad \forall i \in K $ \\
$ \quad \quad \sum_{j \in N} x_{1j} = 1 $ \\
$ \quad \quad \sum_{i \in N} x_{i10} = 1 $ \\
$ \quad \quad x_{ij} \in \{0,1\} \quad \forall (ij) \in A $


### Part a: Implementing model from A9 question 1.

In [ ]:
model2=ConcreteModel()

model2.Nodes=Set(initialize=range(1,11))  
model2.first=1
model2.last=10

model2.Arcs=Set(within=model2.Nodes*model2.Nodes, 
                initialize=[(1,2),(1,6),(1,7),(1,8),
                            (2,1),(6,1),(7,1),(8,1),
                            (2,4),(2,5),(2,7),(2,8),
                            (4,2),(5,2),(7,2),(8,2),
                            (3,6),(3,9),(3,10),
                            (6,3),(9,3),(10,3),
                            (4,5),(4,9),
                            (5,4),(9,4),
                            (5,7),
                            (7,5),
                            (6,8),(6,9),(6,10),
                            (8,6),(9,6),(10,6),
                            (8,9),
                            (9,8)])

model2.Arcs.pprint()

#Add parameter
model2.cost=Param(model2.Arcs,
                  initialize={(1,2):9,(1,6):5,(1,7):17,(1,8):15,
                              (2,1):9,(6,1):5,(7,1):17,(8,1):15,
                              (2,4):4,(2,5):14,(2,7):7,(2,8):6,
                              (4,2):4,(5,2):14,(7,2):7,(8,2):6,
                              (3,6):7,(3,9):2,(3,10):10,
                              (6,3):7,(9,3):2,(10,3):10,
                              (4,5):8,(4,9):11,
                              (5,4):8,(9,4):11,
                              (5,7):4,
                              (7,5):4,
                              (6,8):8,(6,9):12,(6,10):4,
                              (8,6):8,(9,6):12,(10,6):4,
                              (8,9):3,
                              (9,8):3})

#Add dec variables
#arc being used or not - decision variable xij
model2.x=Var(model2.Arcs,domain=Boolean)

In [ ]:
#Adding objective function




In [ ]:
#Adding constraints


### Part b: Solving the model

### Part c: Find the lowest cost from 3 to all 9 remaining nodes.